In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23026396
paper_name = 'zhao_jiang_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [17]:
original_data = pd.read_excel('raw_data/mmc2.xlsx', sheet_name='Sheet1', skiprows=1)

In [18]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 123 x 6


In [19]:
original_data.head()

,&Systemic name,Standard name,Suppression of 0.4M CaCl2 sensitivity by 1 mM MgCl2,Suppression of 0.4M CaCl2 sensitivity by 10 mM MgCl2,Suppression of 0.4M CaCl2 sensitivity by 100 ug/ml CSA,100 ug/ml CSA
0,YAL009W,SPO7,√,√,√,NaN
1,YAL013W,DEP1,√,√,√,NaN
2,YAL024C,LTE1,√,√,NaN,NaN
3,YBL093C,ROX3,√,√,NaN,NaN
4,YBR036C,CSG2,NaN,NaN,NaN,NaN


In [20]:
original_data['orf'] = original_data['&Systemic name'].astype(str)

In [21]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [22]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [23]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                                &Systemic name Standard name  \
index_input                                                                    
120                                                        NaN           NaN   
121          &Gene name in black color indicates the calciu...           NaN   
122          #√ sign indicates the growth phenotype of a mu...           NaN   

            Suppression of 0.4M CaCl2 sensitivity by 1 mM MgCl2  \
index_input                                                       
120                                                        NaN    
121                                                        NaN    
122                                                        NaN    

            Suppression of 0.4M CaCl2 sensitivity by 10 mM MgCl2  \
index_input                                                        
120                                                        NaN     
121                                                        

In [24]:
original_data = original_data.loc[t,:]

In [25]:
original_data['data'] = -1

In [26]:
original_data.set_index('orf', inplace=True)

In [27]:
original_data = original_data[['data']].copy()

In [28]:
original_data = original_data.groupby(original_data.index).mean()

In [29]:
original_data.shape

(120, 1)

# Load & process tested strains

In [32]:
tested = pd.read_excel('raw_data/DELETION LIBRARY.xlsx', sheet_name='DELETION LIBRARY', skiprows=1)

In [33]:
tested.head()

,record no.,ORF name,strain,batch,plate,row,column,RG NOTES,QC NOTES
0,30338,YAL068C,HOM DIP,01_1,301.0,A,2.0,NaN,NaN
1,30339,YAL067C,HOM DIP,01_1,301.0,A,3.0,NaN,NaN
2,30340,YAL066W,HOM DIP,01_1,301.0,A,4.0,NaN,NaN
3,30341,YAL065C,HOM DIP,01_1,301.0,A,5.0,NaN,NaN
4,30345,YAL062W,HOM DIP,01_1,301.0,A,6.0,NaN,NaN


In [34]:
tested['orf'] = tested['ORF name'].astype(str)

In [35]:
tested['orf'] = clean_orf(tested['orf'])

In [38]:
tested.loc[tested['orf']=='YELOO1C','orf'] = 'YEL001C'

In [39]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [40]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             record no. ORF name   strain batch  plate row  column RG NOTES  \
index_input                                                                   
346               30827   YMR41W  HOM DIP  13_5  304.0   F     6.0      NaN   

            QC NOTES     orf  
index_input                   
346              NaN  YMR41W  


In [41]:
tested = tested.loc[t,:]

In [42]:
tested_orfs = tested['orf'].unique()

In [43]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YIR033C']

In [44]:
tested_orfs = list(tested_orfs) + missing

In [45]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [46]:
data = original_data.copy()

In [47]:
dataset_ids = [1309]
datasets = datasets.reindex(index=dataset_ids)

In [48]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [49]:
data.head()

dataset_id,1309
data_type,value
orf,
YAL068C,0
YAL067C,0
YAL066W,0
YAL065C,0
YAL062W,0


## Subset to the genes currently in SGD

In [50]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 25


In [51]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,1309
,data_type,value
gene_id,orf,
1869,YAL068C,0
61,YAL067C,0
60,YAL066W,0
1727,YAL065C,0
57,YAL062W,0


# Normalize

In [52]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [53]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [54]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id       1309       
data_type       value valuez
gene_id orf                 
1869    YAL068C     0    0.0
61      YAL067C     0    0.0
60      YAL066W     0    0.0
1727    YAL065C     0    0.0
57      YAL062W     0    0.0

# Print out

In [55]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [56]:
from IO.save_data_to_db3 import *

In [57]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 23026396...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.54s/it]

Updating the data_modified_on field...
